In [1]:
import os
import pandas as pd
import numpy as np
import collections
import tensorflow as tf
import pickle

In [2]:
number_of_iteration = 3
models_folder_name = os.path.join(os.getcwd(),'models','train', str(number_of_iteration))
summaries_folder_name = os.path.join(os.getcwd(),'summaries','train',str(number_of_iteration))
#path_to_preprocessed_texts = os.path.join(os.getcwd(),
#                                          'texts','preprocessed_texts_for_doc2vec.pkl')

path_to_preprocessed_texts = os.path.join(models_folder_name, 'recipes_train_dataset.pkl')



df_preprocessed_texts = pd.read_pickle(path_to_preprocessed_texts)
preprocessed_texts = df_preprocessed_texts.preprocessed_texts.values.tolist()
labels = df_preprocessed_texts['labels'].values.tolist()

unique_labels=sorted(set(labels))
number_categories=len(unique_labels)
categories_indices=np.linspace(0,number_categories-1,number_categories,dtype=int)
labels2integers=dict(zip(unique_labels,categories_indices))

print(labels2integers)
print("number of recipes in train dataset: ", df_preprocessed_texts.labels.count())

{'sushi': 2, 'steak': 1, 'sashimi': 0, 'tiramisu': 3}
number of recipes in train dataset:  36


In [3]:
batch_size = 8
generations = 150000
model_learning_rate = 0.0005

embedding_size = 24   #word embedding size
doc_embedding_size = 12  #document embedding size
concatenated_size = embedding_size + doc_embedding_size

save_embeddings_every = 5000
print_valid_every = 5000
print_loss_every = 50

In [4]:
def build_dictionary(preprocessed_texts):
    words=[w for words_in_recipe in preprocessed_texts for w in words_in_recipe]
    count = []
    count.extend(collections.Counter(words))
    count=sorted(count)
    word_dict = {}
    for word in count:
        word_dict[word] = len(word_dict)
    
    return (word_dict)

#replace each word in texts with integer value
def text_to_numbers(preprocessed_texts, word_dict):
    data = []
    for prepr_text in preprocessed_texts:
        text_data = []
        for word in prepr_text:
            if word in word_dict:
                word_ix = word_dict[word]
            else:
                word_ix = 0
            text_data.append(word_ix)
        data.append(text_data)
    return (data)


def create_batch_data(text_with_words_conv_to_numbers, batch_size=batch_size):
    batch_data = []
    label_data = []
    
    rand_text_ix = int(np.random.choice(len(text_with_words_conv_to_numbers), size=1))
    rand_text = text_with_words_conv_to_numbers[rand_text_ix]
    word_to_predict_label=np.random.choice(list(set(rand_text)), size=1,replace=False)
    
    while len(batch_data) < batch_size:
        item_in_batch=[]        
        
        label_words=np.random.choice(rand_text, size=1,replace=False)

        item_in_batch.extend(word_to_predict_label)
        item_in_batch.append(rand_text_ix)     
        label_data.extend(label_words)
        batch_data.append(item_in_batch)

        
    batch_data = np.array(batch_data)
    label_data = np.transpose(np.array(label_data))

    return (batch_data, label_data)

In [5]:
word_dictionary=build_dictionary(preprocessed_texts)
vocabulary_size=len(word_dictionary)
print(word_dictionary)
print(vocabulary_size)

word_dictionary_rev = dict(zip(word_dictionary.values(), word_dictionary.keys()))

{'oil': 71, 'ginger': 45, 'guacamole': 50, 'torch': 124, 'sheet': 97, 'pinch': 78, 'mat': 60, 'finger': 40, 'space': 103, 'tuna': 127, 'seed': 95, 'carrot': 17, 'cucumber': 29, 'sugar': 114, 'skewer': 100, 'steak': 110, 'cling': 21, 'filling': 39, 'egg': 36, 'pepper': 76, 'ladyfinger': 52, 'beat': 8, 'ball': 5, 'juice': 51, 'wasabi': 132, 'powder': 81, 'tongs': 122, 'noodle': 69, 'vegetable': 129, 'grill': 48, 'mixture': 67, 'white': 136, 'bit': 10, 'cream': 28, 'drain': 35, 'sashimi': 91, 'dipping': 34, 'filet': 38, 'thumb': 119, 'starch': 109, 'brush': 14, 'coffee': 23, 'asparagus': 2, 'yolk': 139, 'whip': 134, 'cake': 16, 'soy': 102, 'strip': 113, 'metal': 65, 'fryer': 43, 'wine': 137, 'leg': 55, 'flesh': 42, 'cut': 31, 'angle': 1, 'garnish': 44, 'whisk': 135, 'matchstick': 61, 'lime': 56, 'cheese': 18, 'batter': 7, 'vinegar': 130, 'bag': 4, 'worcestershire': 138, 'confectioner': 25, 'mascarpone': 59, 'roll': 88, 'nori': 70, 'mixer': 66, 'salt': 90, 'bottom': 13, 'chive': 19, 'wafer

In [6]:
text_data = text_to_numbers(preprocessed_texts, word_dictionary)
print(text_data)

[[89, 3, 72, 132, 95, 86, 69, 77, 89, 90, 89, 133, 41, 50, 72, 121, 90, 90, 90, 90, 90, 90, 90, 90, 31, 3, 3, 3, 3, 77, 72, 67, 3, 67, 50, 88, 56, 51, 56, 56, 51, 56, 51, 3, 51, 56, 50, 67, 132, 132, 10, 132, 50, 10, 10, 96, 95, 102, 92, 121, 121, 121, 3, 26, 54, 54, 67, 26, 50, 21, 89, 90, 89, 89, 41, 91, 31, 101, 21, 10, 109, 21, 101, 89, 109, 21, 109, 89, 89, 109, 89, 89, 89, 43, 101, 43, 89, 71, 89, 89, 111, 43, 89, 89, 89, 89, 89, 89, 43, 71, 89, 71, 89, 44, 89, 86, 69, 69, 89, 50, 26, 86, 69, 50], [127, 132, 102, 92, 45, 46, 79, 40, 46, 101], [91, 89, 38, 96, 95, 71, 96, 95, 92, 45, 38, 96, 95, 71, 115, 96, 95, 71, 40, 89, 38, 96, 95, 115, 40, 38, 38, 41, 96, 95, 71, 111, 38, 38, 41, 79, 38, 40, 38, 38, 31, 38, 92, 115, 11, 89, 101, 40, 79, 89, 92, 45, 89, 91, 101, 92], [116, 38, 12, 116, 38, 12, 26, 54, 96, 95, 71, 127, 127, 38, 101, 101, 38, 31, 127, 101, 12, 12, 127, 95, 71, 40, 71, 127, 26, 38, 115, 38, 12, 11, 41, 127, 12, 41, 41, 79, 38, 12, 1, 31, 1, 11, 101, 41, 101, 127,

In [7]:
valid_words = ['tuna', 'rice', 'sushi', 'roll', 'sashimi','steak','grill', 'sauce', 'cream']

valid_examples = [word_dictionary[x] for x in valid_words if x in word_dictionary.keys()]
print(valid_examples)

[127, 86, 116, 88, 91, 110, 48, 92, 28]


In [8]:
batch_data, label_data = create_batch_data(preprocessed_texts)
print(batch_data)
print(label_data)
print(np.shape(label_data))

[['steak' '15']
 ['steak' '15']
 ['steak' '15']
 ['steak' '15']
 ['steak' '15']
 ['steak' '15']
 ['steak' '15']
 ['steak' '15']]
['salt' 'chive' 'steak' 'vinegar' 'pepper' 'marinade' 'chive' 'steak']
(8,)


In [9]:
sess = tf.InteractiveSession()

In [10]:
print('Creating Model')

embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0), name="word_embeddings")
doc_embeddings = tf.Variable(tf.random_uniform([len(preprocessed_texts), doc_embedding_size], -1.0, 1.0), name="doc_embeddings")

decoder_weights = tf.Variable(tf.truncated_normal([vocabulary_size, concatenated_size],
                                               stddev=1.0 / np.sqrt(concatenated_size)),
                                               name="decoder_weights")
decoder_biases = tf.Variable(tf.zeros([vocabulary_size]), name="decoder_biases")


x_inputs = tf.placeholder(tf.int32, shape=[None, 2]) #1 for word index and 1 for doc index
y_target = tf.placeholder(tf.int32, shape=[batch_size])
valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

embed= tf.nn.embedding_lookup(embeddings, x_inputs[:, 0])
    
doc_indices = tf.slice(x_inputs, [0,1],[batch_size,1])
doc_embed = tf.nn.embedding_lookup(doc_embeddings,doc_indices)
final_embed = tf.concat([embed, tf.squeeze(doc_embed)],1)

logits = tf.matmul(final_embed, tf.transpose(decoder_weights)) + decoder_biases


loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,labels=y_target))
optimizer = tf.train.AdamOptimizer(learning_rate=model_learning_rate)
train_step = optimizer.minimize(loss)

#cosine similarity between words
norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
normalized_embeddings = embeddings / norm
valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True, name="cosine_similarity")

with tf.name_scope("performance"):
    loss_ph = tf.placeholder(tf.float32,shape=None,name='loss_summary')
    loss_summary = tf.summary.scalar('loss', loss_ph)
performance_summaries = tf.summary.merge([loss_summary])

#saver = tf.train.Saver({"embeddings": embeddings, "doc_embeddings": doc_embeddings})
saver = tf.train.Saver({"embeddings": embeddings, "doc_embeddings": doc_embeddings, "decoder_weights":decoder_weights, "decoder_biases":decoder_biases})
summ_writer = tf.summary.FileWriter(summaries_folder_name, sess.graph)

init = tf.initialize_all_variables()
sess.run(init)


print('Starting Training')

#loss_vec = []
#loss_x_vec = []
for i in range(generations):
    batch_inputs, batch_labels = create_batch_data(text_data)
    feed_dict = {x_inputs : batch_inputs, y_target : batch_labels}

    #run the train step
    sess.run(train_step, feed_dict=feed_dict)

    #return the loss
    if (i+1) % print_loss_every == 0:
        loss_val = sess.run(loss, feed_dict=feed_dict)
        summ = sess.run(performance_summaries, feed_dict={loss_ph:loss_val})
        summ_writer.add_summary(summ, i+1)
        #loss_vec.append(loss_val)
        #loss_x_vec.append(i+1)
        print('Loss at step {} : {}'.format(i+1, loss_val))
        
    #validation
    if (i+1) % print_valid_every == 0:
        sim = sess.run(similarity, feed_dict=feed_dict)
        for j in range(len(valid_words)):
            valid_word = word_dictionary_rev[valid_examples[j]]
            top_k = 5 # number of nearest neighbors
            nearest = (-sim[j, :]).argsort()[1:top_k+1]
            log_str = "Nearest to {}:".format(valid_word)
            for k in range(top_k):
                close_word = word_dictionary_rev[nearest[k]]
                log_str = '{} {},'.format(log_str, close_word)
            print(log_str)
            
    #save dictionary + embeddings
    if (i+1) % save_embeddings_every == 0:
        #save vocabulary dictionary
        with open(os.path.join(models_folder_name,'doc2vec_recipes_dict_words_integers.pkl'), 'wb') as f:
            pickle.dump(word_dictionary, f)
        
        #save embeddings
        model_checkpoint_path = os.path.join(os.getcwd(),models_folder_name,'doc2vec_recipes_checkpoint.ckpt')
        save_path = saver.save(sess, model_checkpoint_path)
        print('Model saved in file: {}'.format(save_path))
        
sess.close()

Creating Model
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Starting Training
Loss at step 50 : 5.089521408081055
Loss at step 100 : 5.371132850646973
Loss at step 150 : 4.668432712554932
Loss at step 200 : 5.138874053955078
Loss at step 250 : 4.8191680908203125
Loss at step 300 : 4.847530364990234
Loss at step 350 : 4.714554786682129
Loss at step 400 : 4.872944355010986
Loss at step 450 : 5.225693702697754
Loss at step 500 : 5.243710994720459
Loss at step 550 : 4.455766677856445
Loss at step 600 : 4.982711315155029
Loss at step 650 : 4.7843546867370605
Loss at step 700 : 5.047980785369873
Loss at step 750 : 5.030146598815918
Loss at step 800 : 5.238272666931152
Loss at step 850 : 4.353737831115723
Loss at step 900 : 4.381399154663086
Loss at step 950 : 5.138124942779541
Loss at step 1000 : 4.170712947845459
Loss at step 1050 : 4.915219306945801
Loss at step 1100 : 5.098161220550537
Lo

Loss at step 9250 : 3.649232864379883
Loss at step 9300 : 2.5338401794433594
Loss at step 9350 : 3.349475145339966
Loss at step 9400 : 1.8582525253295898
Loss at step 9450 : 2.799396276473999
Loss at step 9500 : 3.9648029804229736
Loss at step 9550 : 3.6196727752685547
Loss at step 9600 : 2.914489507675171
Loss at step 9650 : 4.447295188903809
Loss at step 9700 : 3.180666208267212
Loss at step 9750 : 3.7228918075561523
Loss at step 9800 : 2.494056224822998
Loss at step 9850 : 4.779819488525391
Loss at step 9900 : 4.450664520263672
Loss at step 9950 : 3.0552005767822266
Loss at step 10000 : 3.1622233390808105
Nearest to tuna: position, flesh, daikon, sashimi, grate,
Nearest to rice: topping, block, slice, surface, salmon,
Nearest to sushi: mat, wasabi, finger, bamboo, coriander,
Nearest to roll: juice, torch, water, bag, fryer,
Nearest to sashimi: tuna, position, sieve, ground, roe,
Nearest to steak: grate, grill, roe, ground, sashimi,
Nearest to grill: steak, ground, marinade, vinegar,

Loss at step 18100 : 2.009218692779541
Loss at step 18150 : 2.8774986267089844
Loss at step 18200 : 2.599675416946411
Loss at step 18250 : 4.293593406677246
Loss at step 18300 : 3.8443222045898438
Loss at step 18350 : 3.4211983680725098
Loss at step 18400 : 3.4757912158966064
Loss at step 18450 : 3.55869722366333
Loss at step 18500 : 2.2980666160583496
Loss at step 18550 : 3.959000587463379
Loss at step 18600 : 2.450881004333496
Loss at step 18650 : 3.6532139778137207
Loss at step 18700 : 3.323845863342285
Loss at step 18750 : 3.231403350830078
Loss at step 18800 : 3.0325121879577637
Loss at step 18850 : 2.959163188934326
Loss at step 18900 : 2.6065261363983154
Loss at step 18950 : 3.195510149002075
Loss at step 19000 : 2.4377012252807617
Loss at step 19050 : 1.9255459308624268
Loss at step 19100 : 2.0174758434295654
Loss at step 19150 : 4.139527320861816
Loss at step 19200 : 4.095195770263672
Loss at step 19250 : 4.377018928527832
Loss at step 19300 : 2.913100481033325
Loss at step 19

Loss at step 26950 : 2.8426249027252197
Loss at step 27000 : 2.190997838973999
Loss at step 27050 : 2.220132350921631
Loss at step 27100 : 2.0090980529785156
Loss at step 27150 : 2.3487582206726074
Loss at step 27200 : 3.4713969230651855
Loss at step 27250 : 3.023409843444824
Loss at step 27300 : 3.1392760276794434
Loss at step 27350 : 2.8652918338775635
Loss at step 27400 : 3.507373571395874
Loss at step 27450 : 3.3165183067321777
Loss at step 27500 : 4.217070579528809
Loss at step 27550 : 4.0329389572143555
Loss at step 27600 : 3.180821180343628
Loss at step 27650 : 4.037476062774658
Loss at step 27700 : 2.1542317867279053
Loss at step 27750 : 2.9541101455688477
Loss at step 27800 : 2.1044507026672363
Loss at step 27850 : 3.653045654296875
Loss at step 27900 : 2.6445090770721436
Loss at step 27950 : 3.140657424926758
Loss at step 28000 : 3.2582435607910156
Loss at step 28050 : 2.5198564529418945
Loss at step 28100 : 2.9104397296905518
Loss at step 28150 : 3.464503288269043
Loss at st

Loss at step 35850 : 2.510481119155884
Loss at step 35900 : 2.197228193283081
Loss at step 35950 : 4.019049644470215
Loss at step 36000 : 2.589124917984009
Loss at step 36050 : 3.1101531982421875
Loss at step 36100 : 2.6363205909729004
Loss at step 36150 : 2.565025806427002
Loss at step 36200 : 3.290708065032959
Loss at step 36250 : 2.840714931488037
Loss at step 36300 : 2.836008310317993
Loss at step 36350 : 2.8455944061279297
Loss at step 36400 : 2.485311985015869
Loss at step 36450 : 3.121616840362549
Loss at step 36500 : 4.505449295043945
Loss at step 36550 : 3.4273109436035156
Loss at step 36600 : 3.4793219566345215
Loss at step 36650 : 3.039824962615967
Loss at step 36700 : 3.680051326751709
Loss at step 36750 : 2.5760223865509033
Loss at step 36800 : 2.7022790908813477
Loss at step 36850 : 3.4413013458251953
Loss at step 36900 : 2.3608927726745605
Loss at step 36950 : 2.462968349456787
Loss at step 37000 : 2.9383249282836914
Loss at step 37050 : 3.089050769805908
Loss at step 37

...

Loss at step 108400 : 2.0848398208618164
Loss at step 108450 : 2.3084802627563477
Loss at step 108500 : 2.975525140762329
Loss at step 108550 : 3.1846909523010254
Loss at step 108600 : 3.064213752746582
Loss at step 108650 : 2.6032116413116455
Loss at step 108700 : 2.1921956539154053
Loss at step 108750 : 2.848402500152588
Loss at step 108800 : 3.223109722137451
Loss at step 108850 : 3.4826738834381104
Loss at step 108900 : 2.57625675201416
Loss at step 108950 : 1.8792083263397217
Loss at step 109000 : 1.9469335079193115
Loss at step 109050 : 3.0029666423797607
Loss at step 109100 : 3.2528493404388428
Loss at step 109150 : 2.3698477745056152
Loss at step 109200 : 2.7410011291503906
Loss at step 109250 : 2.619145393371582
Loss at step 109300 : 3.3910136222839355
Loss at step 109350 : 2.399054527282715
Loss at step 109400 : 2.921407461166382
Loss at step 109450 : 2.554615020751953
Loss at step 109500 : 3.694849729537964
Loss at step 109550 : 3.2086081504821777
Loss at step 109600 : 2.756

Loss at step 117100 : 2.4730947017669678
Loss at step 117150 : 2.4122138023376465
Loss at step 117200 : 2.6966304779052734
Loss at step 117250 : 3.1598126888275146
Loss at step 117300 : 2.2808499336242676
Loss at step 117350 : 3.5021047592163086
Loss at step 117400 : 3.343390941619873
Loss at step 117450 : 2.1424031257629395
Loss at step 117500 : 2.3209517002105713
Loss at step 117550 : 3.5543582439422607
Loss at step 117600 : 3.3199617862701416
Loss at step 117650 : 2.3815622329711914
Loss at step 117700 : 2.666400909423828
Loss at step 117750 : 2.602346897125244
Loss at step 117800 : 2.860574722290039
Loss at step 117850 : 2.934966564178467
Loss at step 117900 : 2.343893527984619
Loss at step 117950 : 3.2052860260009766
Loss at step 118000 : 2.5860743522644043
Loss at step 118050 : 1.9199939966201782
Loss at step 118100 : 2.1926040649414062
Loss at step 118150 : 2.9700584411621094
Loss at step 118200 : 2.4439964294433594
Loss at step 118250 : 2.647764205932617
Loss at step 118300 : 3

Loss at step 125750 : 2.6168832778930664
Loss at step 125800 : 2.561819076538086
Loss at step 125850 : 3.469022750854492
Loss at step 125900 : 2.579840660095215
Loss at step 125950 : 2.0793581008911133
Loss at step 126000 : 2.8418796062469482
Loss at step 126050 : 3.437980890274048
Loss at step 126100 : 2.5949440002441406
Loss at step 126150 : 2.982875347137451
Loss at step 126200 : 3.9629790782928467
Loss at step 126250 : 2.2363693714141846
Loss at step 126300 : 3.058103322982788
Loss at step 126350 : 2.9096641540527344
Loss at step 126400 : 3.2689194679260254
Loss at step 126450 : 2.999751091003418
Loss at step 126500 : 2.4088375568389893
Loss at step 126550 : 3.511256694793701
Loss at step 126600 : 2.6139168739318848
Loss at step 126650 : 2.649796724319458
Loss at step 126700 : 2.4519379138946533
Loss at step 126750 : 4.096801280975342
Loss at step 126800 : 2.4029364585876465
Loss at step 126850 : 2.9331753253936768
Loss at step 126900 : 2.985381603240967
Loss at step 126950 : 3.221

Loss at step 135050 : 2.7496399879455566
Loss at step 135100 : 2.8176350593566895
Loss at step 135150 : 2.0006027221679688
Loss at step 135200 : 2.703397750854492
Loss at step 135250 : 2.9042506217956543
Loss at step 135300 : 3.518138885498047
Loss at step 135350 : 2.605198621749878
Loss at step 135400 : 3.1075048446655273
Loss at step 135450 : 1.793884038925171
Loss at step 135500 : 2.2500569820404053
Loss at step 135550 : 2.134159564971924
Loss at step 135600 : 2.702195644378662
Loss at step 135650 : 2.6887593269348145
Loss at step 135700 : 2.769425868988037
Loss at step 135750 : 3.108044147491455
Loss at step 135800 : 3.1715450286865234
Loss at step 135850 : 2.3753268718719482
Loss at step 135900 : 3.898165225982666
Loss at step 135950 : 2.5351128578186035
Loss at step 136000 : 2.698298692703247
Loss at step 136050 : 3.061746120452881
Loss at step 136100 : 2.741283893585205
Loss at step 136150 : 2.927743434906006
Loss at step 136200 : 2.786742687225342
Loss at step 136250 : 2.958181

Loss at step 144450 : 2.5048327445983887
Loss at step 144500 : 1.6058177947998047
Loss at step 144550 : 2.5688512325286865
Loss at step 144600 : 2.8496179580688477
Loss at step 144650 : 2.653223991394043
Loss at step 144700 : 3.5360212326049805
Loss at step 144750 : 2.8213844299316406
Loss at step 144800 : 2.6500370502471924
Loss at step 144850 : 3.151747226715088
Loss at step 144900 : 2.591503620147705
Loss at step 144950 : 2.436814308166504
Loss at step 145000 : 2.690741539001465
Nearest to tuna: garnish, position, piece, daikon, flesh,
Nearest to rice: zip, topping, slice, wasabi, surface,
Nearest to sushi: mat, wasabi, finger, coriander, bamboo,
Nearest to roll: mat, cucumber, juice, bag, fryer,
Nearest to sashimi: roe, position, tongs, tomato, sieve,
Nearest to steak: grill, grate, ground, butter, roe,
Nearest to grill: ground, steak, grate, marinade, meat,
Nearest to sauce: soy, salt, starch, vinegar, sieve,
Nearest to cream: cocoa, cheese, espresso, spread, liqueur,
Model saved 